In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['font.size']=25
matplotlib.rcParams['font.family']='serif'

import root_numpy as rn

In [ ]:
rse   =['run','subrun','event']
index_=['run','subrun','event']
FILE_1="/Users/vgenty/sw/larcv/app/LArOpenCVHandle/test/ana_apr17.root"
FILE_2="/Users/vgenty/sw/larcv/app/LArOpenCVHandle/test/ana_apr24.root"

ana_df1  = pd.DataFrame(rn.root2array(FILE_1,treename='tree'))
ana_df2  = pd.DataFrame(rn.root2array(FILE_2,treename='tree'))

eana_df1 = pd.DataFrame(rn.root2array(FILE_1,treename='event_tree'))
eana_df2 = pd.DataFrame(rn.root2array(FILE_2,treename='event_tree'))

eana_df1 = eana_df1.drop_duplicates(index_)
eana_df2 = eana_df2.drop_duplicates(index_)

eana_good_df1 = eana_df1.query("good_croi_ctr>0") 
eana_good_df2 = eana_df2.query("good_croi_ctr>0") 

eana_bad_df1  = eana_df1.query("good_croi_ctr==0") 
eana_bad_df2  = eana_df2.query("good_croi_ctr==0") 

algodata_df1 = pd.DataFrame(rn.root2array(FILE_1,treename='OutTree'))
algodata_df2 = pd.DataFrame(rn.root2array(FILE_2,treename='OutTree'))



In [ ]:
rse_to_entry = eana_df1.reset_index().set_index(rse).entry


In [ ]:
good_df1 = ana_df1.query("scedr<5")
good_df2 = ana_df2.query("scedr<5")

bad_df1  = ana_df1.query("scedr>=5")
bad_df2  = ana_df2.query("scedr>=5")


In [ ]:
good_dfs={}
for example_df in [("before",good_df1),("after",good_df2)]:
    (name,ana_df) = example_df 
    good_entries=[]
    for index, group in ana_df.groupby(rse):
        run,subrun,event=index
        try:
            entry_ = rse_to_entry.ix[(run,subrun,event)]
        except:
            print run,subrun,event
            break
        good_entries.append(entry_)
    good_dfs[name] = good_entries

good_df1_l = good_dfs['before']
good_df2_l = good_dfs['after']

print "Good DF"
print "In 1 not 2"
a = np.unique(np.array([entry for entry in good_df1_l if entry not in good_df2_l]))
print a 
print "sz...",a.size
print 
print "In 2 not 1"
a = np.unique(np.array([entry for entry in good_df2_l if entry not in good_df1_l]))
print a
print "sz...",a.size
print


In [ ]:
res={}
for example_df in [("before",eana_df1,algodata_df1),("after",eana_df2,algodata_df2)]:
    (name,event_df,reco_df) = example_df 
    
    all_entries_v = np.array([i for i in xrange(event_df.index.size)])
    good_croi_entries_v = event_df.query("good_croi_ctr>0").entry.values
    bad_croi_entries_v  = event_df.query("good_croi_ctr==0").entry.values
    good_reco_entries_v = []
    bad_reco_entries_v  = []

    for index, group in event_df.groupby(rse):
        run,subrun,event=index
        xyz_v = reco_df.query("run==@run&subrun==@subrun&event==@event")[['x','y','z']].values
        if xyz_v.shape[0]==0: 
            bad_reco_entries_v.append(group.entry.values[0])
            continue
        sce_v = group[['scex','scey','scez']].values
        try:
            entry_ = rse_to_entry.ix[(run,subrun,event)]
        except:
            print run,subrun,event
            break
        
        if np.where(np.sqrt(np.power(xyz_v-sce_v,2).sum(axis=1))<5)[0].size > 0:
            good_reco_entries_v.append(entry_)
        else:
            bad_reco_entries_v.append(entry_)

    good_reco_entries_v = np.array(good_reco_entries_v)
    good_reco_entries_v.sort()
    
    print "Type:",name
    print "N events...........",all_entries_v.size
    print "N good coi events..",good_croi_entries_v.size
    print "N good reco events.",good_reco_entries_v.size
    print
    res[name]=good_reco_entries_v.copy()
    

In [ ]:
print "before not in after"
print np.array([e for e in res['before'] if e not in res['after']])
print
print "after not in before"
print np.array([e for e in res['after'] if e not in res['before']])
print